In [ ]:
%matplotlib inline
import os
import hs_utils
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from mpl_toolkits.basemap import Basemap
import numpy as np
from osgeo import osr, gdal
import subprocess
data_directory = os.environ['DATA']

In [ ]:
def plot_tiff(tiff, size=(5,5), aspect=1):
    # change the aspect ration to stretch or compress the image
    
    # read the tiff using the gdal library  
    ds = gdal.Open(tiff)
    band = ds.GetRasterBand(1)
    data = band.ReadAsArray()

    # set all negative values (i.e. nodata) to zero so that the map is displayed properly
    data[data<0] = 0

    # create figure to hold plot (figsize=(width, height))
    plt.figure(figsize=size)

    # plot the DEM and display the results
    plt.imshow(data, cmap='gist_earth', aspect=aspect)
    plt.show()

In [ ]:
def run(cmd):
    
    # execute the process
    p = subprocess.Popen(cmd, stdout=subprocess.PIPE, shell=True)
    
    # Grab stdout line by line as it becomes available.  This will loop until p terminates.
    while p.poll() is None:
        l = p.stdout.readline() # This blocks until it receives a newline.
        print(l.strip().decode('utf-8'))
        
    # When the subprocess terminates there might be unconsumed output 
    # that still needs to be processed.
    print(p.stdout.read().strip().decode('utf-8'))
    

In [ ]:
# create a hydroshare intance
h = hs_utils.hydroshare()

In [ ]:
# login to user account (will be replaced with OAUTH soon)
h.getSecureConnection(email = 'TonyCastronova')

In [ ]:
# get some resource content. The resource content is returned as a dictionary
content = h.getResourceContent('1a43f21caa9941d6b79dd53565301175')

In [ ]:
# display the raw dem
raw_dem_path = content['logan.tif']
plot_tiff(raw_dem_path)

In [ ]:
# Fill the DEM Pits

# set the output path for the pitremove operation
fill = os.path.join(data_directory, 'fill.tif')
cmd = 'pitremove -z %s -fel %s' % (raw_dem_path, fill)
print(cmd)
run(cmd)

In [ ]:
# plot the fill result
plot_tiff(fill)

In [ ]:
# Calculate D8 flow direction

# set the output paths for the d8flowdir operation
fdr = os.path.join(data_directory, 'fdr.tif')  # flowdir
sd8 = os.path.join(data_directory, 'sd8.tif')  # slope
cmd = 'd8flowdir -fel %s -sd8 %s -p %s' % (fill, sd8, fdr)
run(cmd)


In [ ]:
# plot the fdr result
plot_tiff(fdr)

In [ ]:
# Calculate D8 contributing area

# set the output paths for the aread8 operation
ad8 = os.path.join(data_directory, 'fdr.tif')  # contributing area
cmd = 'aread8 -p %s -ad8 %s' % (fdr, ad8)
# cmd = 'mpiexec -n 8 aread8 -p %s -ad8 %s' % (fdr, ad8)
run(cmd)

In [ ]:
# plot the contributing area result
plot_tiff(ad8, size=(20,20))

In [ ]:
# Calculate Grid Network

# set the output paths for the gridnet operation
plen = os.path.join(data_directory, 'plen.tif')  # longest flow length upstream
tlen = os.path.join(data_directory, 'tlen.tif')  # total path length upstream
gord = os.path.join(data_directory, 'gord.tif')  # strahler order 
cmd = 'gridnet -p %s -plen %s -tlen %s -gord %s ' % (fdr, plen, tlen, gord)

'''
mpiexec -n 2 gridnet -p /home/jovyan/work/notebooks/data/fdr.tif -plen /home/jovyan/work/n
otebooks/data/plen.tif -tlen /home/jovyan/work/notebooks/data/tlen.tif -gord /home/jovyan/work/notebooks/data/gord.tif             
GridNet version 5.3.5                                                                                                              
Input file /home/jovyan/work/notebooks/data/fdr.tif has projected coordinate system.                                               
This run may take on the order of 1 minutes to complete.                                                                           
This estimate is very approximate.                                                                                                 
Run time is highly uncertain as it depends on the complexity of the input data                                                     
and speed and memory of the computer. This estimate is based on our testing on                                                     
a dual quad core Dell Xeon E5405 2.0GHz PC with 16GB RAM.                                                                          
                                                                                                                                   
===================================================================================                                                
=   BAD TERMINATION OF ONE OF YOUR APPLICATION PROCESSES                                                                           
=   PID 304 RUNNING AT cf07017c04ee                                                                                                
=   EXIT CODE: 11                                                                                                                  
=   CLEANING UP REMAINING PROCESSES                                                                                                
=   YOU CAN IGNORE THE BELOW CLEANUP MESSAGES                                                                                      
===================================================================================                                                
YOUR APPLICATION TERMINATED WITH THE EXIT STRING: Segmentation fault (signal 11)                                                   
This typically refers to a problem with your application.                                                                          
Please see the FAQ page for debugging suggestions                         
'''

print(cmd)
# run('gridnet')
run(cmd)